<a href="https://colab.research.google.com/github/M-Waweru/ObituariesData/blob/master/Obituaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp

%matplotlib inline

In [2]:
# When using Colab, run the code below to load csv

from google.colab import drive
drive.mount('/content/gdrive')

path = "/content/gdrive/My Drive/Colab Notebooks/Obituaries/Obituaries_Dataset.csv"
obituaries_df = pd.read_csv(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
obituaries_df.head()

,Name,Announcement,Death,Burial,Burial_Day,Burial_Week,Gender,Age,Color,Size,Word_Count,No_of_Children,Significant_Children,Significant_Relatives,Fundraising,Death_to_Announce,Death_to_Burial,Announce_to_Burial,No_of_Relatives,County_Burial,County_Death,County_Morgue,Distance_Death,Distance_Morgue,Cause_of_Death,Married,Spouse_Alive,Spouse_gender,Hospital,Morgue,Same_Morgue,Cost_Morgue,Occupation,Repetition,Corporate,Corporate_Name,Residence,Residence_Name,Residence_Category
0,Eng. Paul Chemweno,11/8/2017,11/2/2017,11/17/2017,Friday,Weekday,Male,66.0,Yes,55.18,179,7.0,0.0,0.0,Yes,6,15,9,35,Uasin Gishu,Uasin Gishu,Uasin Gishu,0,0,illness,Yes,Yes,Female,Memorial Hospital Eldoret,MTRH Eldoret,No,NaN,Director,NaN,NaN,NaN,NaN,NaN,NaN
1,Edith Kiende Kiogora,11/8/2017,10/27/2017,11/11/2017,Saturday,Weekend,Female,43.0,Yes,83.60,293,4.0,0.0,7.0,Yes,12,15,3,8,Tharaka Nithi,Nairobi,Tharaka Nithi,224,0,illness,Yes,Yes,Male,Nairobi West Hospital,Chogoria Hospital Mortuary,No,NaN,Teacher,NaN,NaN,NaN,NaN,NaN,NaN
2,Simon Muchiri Ngigii Mbaria,11/8/2017,11/1/2017,11/8/2017,Wednesday,Weekday,Male,35.0,Yes,109.12,206,4.0,0.0,2.0,No,7,7,0,7,Nyandarua,Nyandarua,Nakuru,0,77,illness,Yes,Yes,Female,Charity Hospital Nyahururu,Umash Nakuru,No,NaN,Teacher,NaN,NaN,NaN,NaN,NaN,NaN
3,Mwalimu Erick Namusasi Werunga,11/8/2017,10/24/2017,11/9/2017,Thursday,Weekday,Male,54.0,Yes,83.60,171,4.0,0.0,0.0,No,15,16,1,33,Trans Nzioa,NaN,Trans Nzioa,0,NaN,illness,Yes,No,Female,NaN,Maida Mortuary,NaN,NaN,Teacher,NaN,NaN,NaN,NaN,NaN,NaN
4,Catherine Mueni Sina,11/8/2017,11/2/2017,11/11/2017,Saturday,Weekend,Female,41.0,No,55.18,150,1.0,0.0,NaN,NaN,6,9,3,17,Machakos,Uasin Gishu,NaN,378,NaN,accident,NaN,NaN,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# EDA

In [4]:
obituaries_df.columns

Index(['Name', 'Announcement', 'Death', 'Burial', 'Burial_Day', 'Burial_Week',
       'Gender', 'Age', 'Color', 'Size', 'Word_Count', 'No_of_Children',
       'Significant_Children', 'Significant_Relatives', 'Fundraising',
       'Death_to_Announce', 'Death_to_Burial', 'Announce_to_Burial',
       'No_of_Relatives', 'County_Burial', 'County_Death', 'County_Morgue',
       'Distance_Death', 'Distance_Morgue', 'Cause_of_Death', 'Married',
       'Spouse_Alive', 'Spouse_gender', 'Hospital', 'Morgue', 'Same_Morgue',
       'Cost_Morgue', 'Occupation', 'Repetition', 'Corporate',
       'Corporate_Name', 'Residence', 'Residence_Name', 'Residence_Category'],
      dtype='object')

In [5]:
obituaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1291 entries, 0 to 1290
Data columns (total 39 columns):
Name                     1291 non-null object
Announcement             1265 non-null object
Death                    893 non-null object
Burial                   1164 non-null object
Burial_Day               1163 non-null object
Burial_Week              1163 non-null object
Gender                   1291 non-null object
Age                      507 non-null float64
Color                    1289 non-null object
Size                     1237 non-null float64
Word_Count               1237 non-null object
No_of_Children           1093 non-null float64
Significant_Children     1092 non-null float64
Significant_Relatives    1083 non-null float64
Fundraising              1142 non-null object
Death_to_Announce        1291 non-null object
Death_to_Burial          1244 non-null object
Announce_to_Burial       1278 non-null object
No_of_Relatives          1061 non-null object
County_Burial   

In [6]:
obituaries_df['Burial_Day'].value_counts()

Saturday     373
Friday       364
Thursday     137
Tuesday      111
Wednesday    101
Monday        50
Sunday        16
saturday      11
Name: Burial_Day, dtype: int64

In [7]:
obituaries_df['Burial_Day'].unique()

array(['Friday', 'Saturday', 'Wednesday', 'Thursday', 'Tuesday', 'Monday',
       nan, 'Sunday', 'saturday'], dtype=object)

In [8]:
pp.ProfileReport(obituaries_df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,39
Number of observations,1291
Total Missing (%),38.0%
Total size in memory,393.5 KiB
Average record size in memory,312.1 B
Numeric,5
Categorical,34
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [9]:
obituaries_df.isnull().sum()

Name                        0
Announcement               26
Death                     398
Burial                    127
Burial_Day                128
Burial_Week               128
Gender                      0
Age                       784
Color                       2
Size                       54
Word_Count                 54
No_of_Children            198
Significant_Children      199
Significant_Relatives     208
Fundraising               149
Death_to_Announce           0
Death_to_Burial            47
Announce_to_Burial         13
No_of_Relatives           230
County_Burial             247
County_Death              766
County_Morgue             411
Distance_Death            805
Distance_Morgue           420
Cause_of_Death            493
Married                   243
Spouse_Alive              348
Spouse_gender             356
Hospital                 1024
Morgue                    426
Same_Morgue              1088
Cost_Morgue              1289
Occupation               1040
Repetition

In [10]:
obituaries_df.duplicated().sum()

0

# Data Preparation and Analysis 

In [11]:
obituaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1291 entries, 0 to 1290
Data columns (total 39 columns):
Name                     1291 non-null object
Announcement             1265 non-null object
Death                    893 non-null object
Burial                   1164 non-null object
Burial_Day               1163 non-null object
Burial_Week              1163 non-null object
Gender                   1291 non-null object
Age                      507 non-null float64
Color                    1289 non-null object
Size                     1237 non-null float64
Word_Count               1237 non-null object
No_of_Children           1093 non-null float64
Significant_Children     1092 non-null float64
Significant_Relatives    1083 non-null float64
Fundraising              1142 non-null object
Death_to_Announce        1291 non-null object
Death_to_Burial          1244 non-null object
Announce_to_Burial       1278 non-null object
No_of_Relatives          1061 non-null object
County_Burial   

In [12]:
obituaries_df.sample(10)

,Name,Announcement,Death,Burial,Burial_Day,Burial_Week,Gender,Age,Color,Size,Word_Count,No_of_Children,Significant_Children,Significant_Relatives,Fundraising,Death_to_Announce,Death_to_Burial,Announce_to_Burial,No_of_Relatives,County_Burial,County_Death,County_Morgue,Distance_Death,Distance_Morgue,Cause_of_Death,Married,Spouse_Alive,Spouse_gender,Hospital,Morgue,Same_Morgue,Cost_Morgue,Occupation,Repetition,Corporate,Corporate_Name,Residence,Residence_Name,Residence_Category
1022,Tobias Odongo Obura,2/15/2017,NaN,2/17/2017,Friday,Weekday,Male,NaN,Yes,112.34,210,10.0,0.0,0.0,Yes,"42,781.00","42,783",2,24,Siaya,Siaya,Siaya,0,0,illness,Yes,Yes,Female,NaN,Ahero sub county hospital mortuary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,Benjamin Kamau Kingori,5/31/2017,5/26/2017,2/6/2017,Friday,Weekday,Male,66.0,Yes,85.44,154,3.0,0.0,0.0,No,#VALUE!,7,2,24,Nyeri,NaN,Nyeri,NaN,0,illness,Yes,Yes,Female,NaN,Mathari Mission Hospital Mortuary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
490,Goerge Munyinyi Mburu,10/3/2017,5/3/2017,3/13/2017,Monday,Weekday,Male,60.0,Yes,68.53,133,4.0,0.0,2.0,No,153,8,3,8,Nyandarua,NaN,Nyandarua,NaN,0,illness,Yes,Yes,Female,NaN,Engineer Hospital Mortuary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,Hellen Amunga Oronje,10/31/2017,10/26/2017,11/4/2017,Saturday,Weekend,Female,NaN,No,84.48,134,4.0,0.0,0.0,Yes,5,9,4,8,Nairobi,Nairobi,NaN,NaN,NaN,illness,NaN,NaN,NaN,Nairobi Women's Hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,Tom lempaa,5/11/2017,2/5/2017,12/5/2017,Friday,Weekday,Male,77.0,yes,158.40,323,9.0,0.0,61.0,No,95,9,10,1,Narok,Narok,Narok,0,0,NaN,yes,Yes,Female,NaN,Narok Hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,Jane Wanjiku Waruingi,11/15/2017,NaN,11/17/2017,Friday,Weekday,Female,NaN,Yes,84.48,201,NaN,NaN,NaN,NaN,"43,054.00",NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,Rev. Francis Mwangi Muchungu,1/7/2017,NaN,1/9/2017,NaN,NaN,Male,NaN,Yes,54.56,146,6.0,1.0,1.0,No,"42,742.00",NaN,2,7,Laikipia,Laikipia,Laikipia,0,0,NaN,Yes,Yes,Female,NaN,Nanyuki General Hospital Mortuary,NaN,NaN,Pastor,0,No,None,NaN,NaN,NaN
540,Emma Nyambura Kamau,5/24/2017,5/17/2017,5/25/2017,Thursday,Weekday,Female,NaN,Yes,56.07,210,3.0,0.0,3.0,No,#VALUE!,8,1,36,Kiambu,NaN,Nairobi,NaN,32,illness,Yes,Yes,Male,NaN,Kenyatta University,NaN,NaN,public servant,NaN,Yes,County Government of Kiambu,NaN,NaN,NaN
1213,Bartholomew Ngiyo M'bwika,2/23/2017,2/19/2017,2/25/2017,Saturday,Weekend,Male,NaN,Yes,67.76,300,5.0,3.0,6.0,Yes,4,6,2,31,Meru,Meru,Meru,0,0,illness,Yes,Yes,Female,Maua Methodist Hospital,Maua Methodist Hospital Mortuary,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
912,Rodgers Tsilwana,5/10/2017,2/5/2017,10/5/2017,Wednesday,Weekday,Male,39.0,no,68.64,145,3.0,0.0,12.0,Yes,94,8,8,0,Vihiga,NaN,Vihiga,NaN,0,NaN,yes,Yes,Female,NaN,Vihiga county mortuary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Create copy for manipulation purposes
obituaries_copy_df = obituaries_df.copy()

In [14]:
obituaries_copy_df.columns

Index(['Name', 'Announcement', 'Death', 'Burial', 'Burial_Day', 'Burial_Week',
       'Gender', 'Age', 'Color', 'Size', 'Word_Count', 'No_of_Children',
       'Significant_Children', 'Significant_Relatives', 'Fundraising',
       'Death_to_Announce', 'Death_to_Burial', 'Announce_to_Burial',
       'No_of_Relatives', 'County_Burial', 'County_Death', 'County_Morgue',
       'Distance_Death', 'Distance_Morgue', 'Cause_of_Death', 'Married',
       'Spouse_Alive', 'Spouse_gender', 'Hospital', 'Morgue', 'Same_Morgue',
       'Cost_Morgue', 'Occupation', 'Repetition', 'Corporate',
       'Corporate_Name', 'Residence', 'Residence_Name', 'Residence_Category'],
      dtype='object')

In [0]:
# Columns to delete
columns_to_drop = ['Hospital', 'Morgue', 'Same_Morgue', 'Cost_Morgue', 'Occupation', 'Repetition', 'Corporate', 'Corporate_Name', 'Residence', 'Residence_Name', 'Residence_Category']

def deleteCols(df, cols):
  df.drop(columns=cols, inplace=True)
  return df

# Columns for dates to parse
columns_to_parse = ['Announcement', 'Death', 'Burial']

def parseTheDates(df, cols):
  for col in cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')
  return df

# Columns to change from object to float
columns_to_float = ['Word_Count', 'Death_to_Announce', 'Death_to_Burial', 'Announce_to_Burial', 'No_of_Relatives',
                    'Distance_Death', 'Distance_Morgue']   

def changeDataTypes(df, cols):
  for col in cols:
    df.loc[df[col] == '#VALUE!', col] = np.nan 
    df[col] = pd.to_numeric(df[col].str.replace(',', ''), errors='coerce')
    print(col)
  return df

# changeDataTypes(obituaries_copy_df, ['Distance_Morgue']).sample(10)

In [16]:
# jhkjh
prepared_df = (obituaries_copy_df.pipe(deleteCols, cols=columns_to_drop)
                                .pipe(parseTheDates, cols=columns_to_parse)
                                .pipe(changeDataTypes, cols=columns_to_float))
prepared_df.head()

Word_Count
Death_to_Announce
Death_to_Burial
Announce_to_Burial
No_of_Relatives
Distance_Death
Distance_Morgue


,Name,Announcement,Death,Burial,Burial_Day,Burial_Week,Gender,Age,Color,Size,Word_Count,No_of_Children,Significant_Children,Significant_Relatives,Fundraising,Death_to_Announce,Death_to_Burial,Announce_to_Burial,No_of_Relatives,County_Burial,County_Death,County_Morgue,Distance_Death,Distance_Morgue,Cause_of_Death,Married,Spouse_Alive,Spouse_gender
0,Eng. Paul Chemweno,2017-11-08,2017-11-02,2017-11-17,Friday,Weekday,Male,66.0,Yes,55.18,179.0,7.0,0.0,0.0,Yes,6.0,15.0,9.0,35.0,Uasin Gishu,Uasin Gishu,Uasin Gishu,0.0,0.0,illness,Yes,Yes,Female
1,Edith Kiende Kiogora,2017-11-08,2017-10-27,2017-11-11,Saturday,Weekend,Female,43.0,Yes,83.60,293.0,4.0,0.0,7.0,Yes,12.0,15.0,3.0,8.0,Tharaka Nithi,Nairobi,Tharaka Nithi,224.0,0.0,illness,Yes,Yes,Male
2,Simon Muchiri Ngigii Mbaria,2017-11-08,2017-11-01,2017-11-08,Wednesday,Weekday,Male,35.0,Yes,109.12,206.0,4.0,0.0,2.0,No,7.0,7.0,0.0,7.0,Nyandarua,Nyandarua,Nakuru,0.0,77.0,illness,Yes,Yes,Female
3,Mwalimu Erick Namusasi Werunga,2017-11-08,2017-10-24,2017-11-09,Thursday,Weekday,Male,54.0,Yes,83.60,171.0,4.0,0.0,0.0,No,15.0,16.0,1.0,33.0,Trans Nzioa,NaN,Trans Nzioa,0.0,NaN,illness,Yes,No,Female
4,Catherine Mueni Sina,2017-11-08,2017-11-02,2017-11-11,Saturday,Weekend,Female,41.0,No,55.18,150.0,1.0,0.0,NaN,NaN,6.0,9.0,3.0,17.0,Machakos,Uasin Gishu,NaN,378.0,NaN,accident,NaN,NaN,Male


In [17]:
obituaries_copy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1291 entries, 0 to 1290
Data columns (total 28 columns):
Name                     1291 non-null object
Announcement             1263 non-null datetime64[ns]
Death                    887 non-null datetime64[ns]
Burial                   1164 non-null datetime64[ns]
Burial_Day               1163 non-null object
Burial_Week              1163 non-null object
Gender                   1291 non-null object
Age                      507 non-null float64
Color                    1289 non-null object
Size                     1237 non-null float64
Word_Count               1237 non-null float64
No_of_Children           1093 non-null float64
Significant_Children     1092 non-null float64
Significant_Relatives    1083 non-null float64
Fundraising              1142 non-null object
Death_to_Announce        819 non-null float64
Death_to_Burial          1168 non-null float64
Announce_to_Burial       1234 non-null float64
No_of_Relatives          1060 non-n